# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chemin grenier,-20.4872,57.4656,74.79,87,36,7.87,MU,1732491771
1,1,blackmans bay,-43.0167,147.3167,57.33,94,94,6.02,AU,1732491771
2,2,mersin,36.8620,34.6509,43.52,45,2,8.08,TR,1732491488
3,3,albany,42.6001,-73.9662,38.53,66,71,3.00,US,1732491562
4,4,zeya,53.7500,127.2667,12.00,93,100,2.91,RU,1732491567


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols = ["City", "Humidity"]
)

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] < 80.6) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

2


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
176,176,salalah,17.0151,54.0924,77.09,73,0,3.44,OM,1732491788
532,532,berbera,10.4396,45.0143,79.59,67,0,4.05,SO,1732491822


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
176,salalah,OM,17.0151,54.0924,73,
532,berbera,SO,10.4396,45.0143,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salalah - nearest hotel: Muscat International Hotel
berbera - nearest hotel: Al Madina Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
176,salalah,OM,17.0151,54.0924,73,Muscat International Hotel
532,berbera,SO,10.4396,45.0143,67,Al Madina Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
hotelPlot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotelPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)